In [ ]:
'''
Exercise in Collaborative Filtering using data from MovieLens + IMDB/RottenTomatoes

Process:
    1. Import and clean data 
    2. Create matrix for movie ratings for each user/movie pair
    3. Find K most similar users to a selected user (aka User)
        a. Use KNN algorithm
        b. Use Hamming distance to compare vectors (# mismatched dimension values / # of total dimensions)
    4. Using K most similar users, calculate mean rating of each movie
    5. Select top N movies with greatest mean that have not been rated by User
       (These are the top N movies to recommend to User)
    6. Get details for each of the N movies
    7. Analyze movie details to look for user preferences


'''

In [1]:
import datetime
import numpy as np
import pandas as pd
from scipy.spatial.distance import hamming

# 1) Import and clean data
def file_to_df(path, usecols, delimiter=','):
    '''
    Import text file data to a pandas dataframe object
    
    param path: path of text file containing data to import
    param usecols: list of columns to take
    param delimiter: character used to separate data elements in a line
    
    return: DataFrame object of data frm text file
    '''
    # Read file line-by-line into a list
    with open(path, errors='replace') as file:
        data = file.readlines()
    
    # Clean data
    for i in range(len(data)):
        data[i] = data[i].replace('\n', '') # Remove newline chars
    
    # Convert array of strings into 2D list of strings
    data = [line.split(delimiter) for line in data]
    
    # Convert 2D list to a DataFrame object
    df = pd.DataFrame(data)
    
    # Set column names to values in first row
    df.columns = list(df.iloc[0])

    # Take specified columns only
    df = df[usecols]
    
    # Delete first row since it contains column names and not values
    df = df.drop(0)
    
    # Reset to 0-based index
    df.index = range(len(df.index))
    
    return df


# Set data file paths
movies_path = 'data/movielens_2k/movies.dat'
movie_actors_path = 'data/movielens_2k/movie_actors.dat'
movie_directors_path = 'data/movielens_2k/movie_directors.dat'
movie_genres_path = 'data/movielens_2k/movie_genres.dat'
user_rated_movies_path = 'data/movielens_2k/user_ratedmovies.dat'

# Import data, clean it, and output to dataframes
df_movies = file_to_df(movies_path, delimiter='\t', usecols=['id','title','imdbID','rtID','rtAllCriticsRating','rtAllCriticsNumReviews','rtAllCriticsScore','rtTopCriticsRating','rtTopCriticsNumReviews','rtTopCriticsNumFresh','rtTopCriticsNumRotten','rtTopCriticsScore','rtAudienceRating','rtAudienceNumRatings','rtAudienceScore'])
df_movie_actors = file_to_df(movie_actors_path, delimiter='\t', usecols=['movieID','actorID','actorName','ranking'])
df_movie_directors = file_to_df(movie_directors_path, delimiter='\t', usecols=['movieID','directorID','directorName'])
df_movie_genres = file_to_df(movie_genres_path, delimiter='\t', usecols=['movieID','genre'])
df_user_rated_movies = file_to_df(user_rated_movies_path, delimiter='\t', usecols=['userID','movieID','rating']) 

# Create User/Movie ratings matrix
df_user_movie_ratings = df_user_rated_movies.pivot(index='userID', columns='movieID', values='rating')

# Replace null values with a numerical equivalent, then cast all data types in table to float 
df_user_movie_ratings.fillna(value=np.nan,inplace=True)
df_user_movie_ratings = pd.DataFrame(df_user_movie_ratings, dtype=np.float32)


In [2]:
# User movie ratings dataframe preview 
print(str.format('Users: {}'.format(df_user_movie_ratings.shape[0])))
print(str.format('Movies: {}'.format(df_user_movie_ratings.shape[1])))
df_user_movie_ratings.head()

Users: 2113
Movies: 10109


movieID,1,10,100,1000,1001,1002,1003,1004,1005,1006,...,99,990,991,992,993,994,996,997,998,999
userID,,,,,,,,,,,,,,,,,,,,,
10025,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10058,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10064,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10084,4.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10094,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
def hamming_distance(userID_1, userID_2):
    '''
    Finds hamming distance between two users.
    Hamming distance is the number of features that differ between the two users.
    
    '''
    distance = hamming(df_user_movie_ratings.loc[userID_1], df_user_movie_ratings.loc[userID_2])
    return distance

def find_knn(userID, k=3):
    '''
    Finds k nearest neighbors of userID based on hamming distance
    
    param userID: userID to get neighbors for
    param k: number of neighbors
    '''
    # Get data for all other users except specified user
    df_other_user_movie_ratings = df_user_movie_ratings.loc[df_user_movie_ratings.index != userID]
    
    # Calculate distance between specified user and all other users
    df_other_user_movie_ratings['distance'] = df_other_user_movie_ratings.apply(lambda row: hamming_distance(userID,row.name), axis=1)
    
    # Sort users by distance (closest first)
    df_other_user_movie_ratings.sort_values(['distance'],axis=0,inplace=True)
    
    # Take top k users (k nearest users)
    df_other_user_movie_ratings = df_other_user_movie_ratings.iloc[0:k]
    
    return df_other_user_movie_ratings

###
print(datetime.datetime.now())
userID = '10058'
#df_user_movie_ratings = df_user_movie_ratings.iloc[0:11]
df_knn = find_knn(userID,k=10)
print(datetime.datetime.now())

2018-02-04 09:33:35.285605
2018-02-04 09:33:37.804366


//anaconda/lib/python3.5/site-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
//anaconda/lib/python3.5/site-packages/ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [4]:
def get_unrated_movies_for_user(userID):
    '''
    Gets unrated movies from user/movie ratings matrix by finding row of userID, 
    then finding any column (movieID) with a null value
    
    param userID: userID to get unrated movies for
    
    return: list of movieID's that the specified user has not rated yet
    '''
    
    # n = single user's movie ratings
    n = df_user_movie_ratings.loc[df_user_movie_ratings.index == userID]
    cols = list(n.columns)
    unrated_movies = [cols[i] for i in range(len(cols)) if pd.isnull(n[str(cols[i])][0])]
    
    return unrated_movies

###
print(datetime.datetime.now())
# Get mean of movie ratings for nearest neighbors
# avg = df_knn.fillna(value=0).agg(['mean'], numeric_only=True)
avg = df_knn.agg(['mean'], numeric_only=True)

# Place avg movie ratings into a column, then remove any movies rated by the user
unrated_movies = get_unrated_movies_for_user(userID)
avg_filtered = avg.transpose().loc[unrated_movies]

# Sort movies based on mean average (descending)
avg_sorted = avg_filtered.sort_values(['mean'],axis=0,ascending=False)
print(datetime.datetime.now())

2018-02-04 09:33:40.600710
2018-02-04 09:33:50.419901


In [5]:
top_recommendations = avg_sorted[0:10]
top_recommendations

,mean
8684,5.0
921,5.0
3811,5.0
3814,5.0
3822,5.0
6609,5.0
8256,5.0
8197,5.0
7938,5.0
4658,5.0


In [6]:
def get_movie_details(movie_id):
    movie_details = {}
    movie_details['title'] = [df_movies.loc[df_movies.id == movie_id].iloc[0].title]
    movie_details['actors'] = list(df_movie_actors.loc[df_movie_actors.movieID == movie_id].actorName)
    movie_details['director'] = [df_movie_directors.loc[df_movie_directors.movieID == movie_id].iloc[0].directorName]
    movie_details['genres'] = list(df_movie_genres.loc[df_movie_genres.movieID == movie_id].genre)
    
    return movie_details

def movie_details_to_df(movie_details):
    data = [movie_details['title'],
            movie_details['director'],
            movie_details['actors'],
            movie_details['genres']] 
    
    df = pd.DataFrame(data).transpose().fillna(value='')
    df.columns=['title','director','actors','genres']
    
    return df

# Get list of all movie details after converting each set of movie details to a dataframe
movie_details_df_list = [movie_details_to_df(get_movie_details(movieID)) for movieID in list(top_recommendations.index)]

# Concatenate all movies into a single dataframe for futher analysis
df_movie_details = pd.concat(movie_details_df_list)

# Create an ID column
df_movie_details['id'] = range(df_movie_details.shape[0])

In [6]:
# Look for favorite genres
print(df_movie_details.groupby('genres')['title'].count().drop('').sort_values(ascending=False))

# look for favorite actors
print(df_movie_details.groupby('actors')['id'].count().sort_values(ascending=False).iloc[0:10])

# look for favorite directors
print(df_movie_details.groupby('director')['id'].count().drop('').sort_values(ascending=False))

genres
Drama        8
Romance      3
War          2
Comedy       2
Thriller     1
Mystery      1
Horror       1
Adventure    1
Name: title, dtype: int64
actors
Jessica Harper        2
Fred Kohler Jr.       1
Frank Wilson          1
Frank Adu             1
Francesco Leonetti    1
Franca Cupane         1
Fox Harris            1
Florian               1
Ferruccio Nuzzo       1
Ferdinand Munier      1
Name: id, dtype: int64
director
Woody Allen             1
Rouben Mamoulian        1
Robert Bresson          1
Richard Benjamin        1
Pier Paolo Pasolini     1
Patrice Leconte         1
Ingmar Bergman          1
Bruce Beresford         1
Alejandro Jodorowsky    1
Alain Resnais           1
Name: id, dtype: int64


In [38]:
'''
for index, row in df.iterrows():
    print row['c1'], row['c2']
'''
# Look for unique titles
titles = pd.Series(df_movie_details['title'].unique())
titles = [value for index, value in titles.iteritems() if value != '']

# Look for favorite genres
genres = df_movie_details.groupby('genres')['title'].count().drop('').sort_values(ascending=False)
genres = [index + ', ' + str(value) for index, value in genres.iteritems()]

# look for favorite actors
actors = df_movie_details.groupby('actors')['id'].count().sort_values(ascending=False).iloc[0:10]
actors = [index + ', ' + str(value) for index, value in actors.iteritems()]

# look for favorite directors
directors = df_movie_details.groupby('director')['id'].count().drop('').sort_values(ascending=False)
directors = [index + ', ' + str(value) for index, value in directors.iteritems()]

results = {
    'titles': titles,
    'stats':{
        'genres': genres,
        'actors': actors,
        'directors': directors
    }
}

In [39]:
results

{'stats': {'actors': ['Jessica Harper, 2',
   'Fred Kohler Jr., 1',
   'Frank Wilson, 1',
   'Frank Adu, 1',
   'Francesco Leonetti, 1',
   'Franca Cupane, 1',
   'Fox Harris, 1',
   'Florian, 1',
   'Ferruccio Nuzzo, 1',
   'Ferdinand Munier, 1'],
  'directors': ['Woody Allen, 1',
   'Rouben Mamoulian, 1',
   'Robert Bresson, 1',
   'Richard Benjamin, 1',
   'Pier Paolo Pasolini, 1',
   'Patrice Leconte, 1',
   'Ingmar Bergman, 1',
   'Bruce Beresford, 1',
   'Alejandro Jodorowsky, 1',
   'Alain Resnais, 1'],
  'genres': ['Drama, 8',
   'Romance, 3',
   'War, 2',
   'Comedy, 2',
   'Thriller, 1',
   'Mystery, 1',
   'Horror, 1',
   'Adventure, 1']},
 'titles': ["Un condamn� � mort s'est �chapp� ou Le vent souffle o� il veut",
  'My Favorite Year',
  "'Breaker' Morant",
  'Love and Death',
  'La fille sur le pont',
  'Il vangelo secondo Matteo',
  'Queen Christina',
  'Hiroshima mon amour',
  'Nattvardsg�sterna',
  'Santa sangre']}

In [37]:
import json
json.dumps(results)

'{"genres": ["Drama, 8", "Romance, 3", "War, 2", "Comedy, 2", "Thriller, 1", "Mystery, 1", "Horror, 1", "Adventure, 1"], "titles": ["Un condamn\\ufffd \\ufffd mort s\'est \\ufffdchapp\\ufffd ou Le vent souffle o\\ufffd il veut", "My Favorite Year", "\'Breaker\' Morant", "Love and Death", "La fille sur le pont", "Il vangelo secondo Matteo", "Queen Christina", "Hiroshima mon amour", "Nattvardsg\\ufffdsterna", "Santa sangre"], "actors": ["Jessica Harper, 2", "Fred Kohler Jr., 1", "Frank Wilson, 1", "Frank Adu, 1", "Francesco Leonetti, 1", "Franca Cupane, 1", "Fox Harris, 1", "Florian, 1", "Ferruccio Nuzzo, 1", "Ferdinand Munier, 1"], "directors": ["Woody Allen, 1", "Rouben Mamoulian, 1", "Robert Bresson, 1", "Richard Benjamin, 1", "Pier Paolo Pasolini, 1", "Patrice Leconte, 1", "Ingmar Bergman, 1", "Bruce Beresford, 1", "Alejandro Jodorowsky, 1", "Alain Resnais, 1"]}'

In [41]:
import datetime
a = datetime.datetime.now()

In [42]:
b = datetime.datetime.now()

In [45]:
print(str(results))

{'stats': {'genres': ['Drama, 8', 'Romance, 3', 'War, 2', 'Comedy, 2', 'Thriller, 1', 'Mystery, 1', 'Horror, 1', 'Adventure, 1'], 'directors': ['Woody Allen, 1', 'Rouben Mamoulian, 1', 'Robert Bresson, 1', 'Richard Benjamin, 1', 'Pier Paolo Pasolini, 1', 'Patrice Leconte, 1', 'Ingmar Bergman, 1', 'Bruce Beresford, 1', 'Alejandro Jodorowsky, 1', 'Alain Resnais, 1'], 'actors': ['Jessica Harper, 2', 'Fred Kohler Jr., 1', 'Frank Wilson, 1', 'Frank Adu, 1', 'Francesco Leonetti, 1', 'Franca Cupane, 1', 'Fox Harris, 1', 'Florian, 1', 'Ferruccio Nuzzo, 1', 'Ferdinand Munier, 1']}, 'titles': ["Un condamn� � mort s'est �chapp� ou Le vent souffle o� il veut", 'My Favorite Year', "'Breaker' Morant", 'Love and Death', 'La fille sur le pont', 'Il vangelo secondo Matteo', 'Queen Christina', 'Hiroshima mon amour', 'Nattvardsg�sterna', 'Santa sangre']}


In [44]:
'Time to process request: ' + str(b-a)

'Time to process request: 0:00:08.200070'

In [47]:
json.dumps(results)

'{"stats": {"genres": ["Drama, 8", "Romance, 3", "War, 2", "Comedy, 2", "Thriller, 1", "Mystery, 1", "Horror, 1", "Adventure, 1"], "directors": ["Woody Allen, 1", "Rouben Mamoulian, 1", "Robert Bresson, 1", "Richard Benjamin, 1", "Pier Paolo Pasolini, 1", "Patrice Leconte, 1", "Ingmar Bergman, 1", "Bruce Beresford, 1", "Alejandro Jodorowsky, 1", "Alain Resnais, 1"], "actors": ["Jessica Harper, 2", "Fred Kohler Jr., 1", "Frank Wilson, 1", "Frank Adu, 1", "Francesco Leonetti, 1", "Franca Cupane, 1", "Fox Harris, 1", "Florian, 1", "Ferruccio Nuzzo, 1", "Ferdinand Munier, 1"]}, "titles": ["Un condamn\\ufffd \\ufffd mort s\'est \\ufffdchapp\\ufffd ou Le vent souffle o\\ufffd il veut", "My Favorite Year", "\'Breaker\' Morant", "Love and Death", "La fille sur le pont", "Il vangelo secondo Matteo", "Queen Christina", "Hiroshima mon amour", "Nattvardsg\\ufffdsterna", "Santa sangre"]}'

In [48]:
df_user_movie_ratings.shape

(2113, 10109)

In [55]:
df_user_rated_movies.groupby('userID')['userID'].count().sort_values(ascending=False).iloc[0:10]

userID
6757     3410
30687    2908
30500    2823
62332    2763
51033    2631
58087    2529
1860     2528
53315    2517
30167    2451
31854    2342
Name: userID, dtype: int64

In [54]:
df_user_rated_movies

,userID,movieID,rating
0,75,3,1
1,75,32,4.5
2,75,110,4
3,75,160,2
4,75,163,4
5,75,165,4.5
6,75,173,3.5
7,75,296,5
8,75,353,3.5
9,75,420,2


In [56]:
# UserID = 10058
df_user_rated_movies.groupby('userID')['userID'].count().sort_values(ascending=False).loc['10058']

113

In [57]:
# UserID = 30500
df_user_rated_movies.groupby('userID')['userID'].count().sort_values(ascending=False).loc['30500']

2823